#### Objetivo

 * Aplicar los conceptos de regresión lineal para resolver un problema práctico utilizando Python.

In [2]:
# Se cargan las bibliotecas
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Se usa biblioteca google para poder usar archivos .
from google.colab import drive
# Se conecta colab con drive.
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Se carga el conjunto de datos
path ="/content/drive/MyDrive/CLASE MACHINE LEARNING/DATA_CORES/house-prices.csv"
df = pd.read_csv(path)

#### Exploración de datos

In [5]:
df.head()

,Home,Price,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood
0,1,114300,1790,2,2,2,No,East
1,2,114200,2030,4,2,3,No,East
2,3,114800,1740,3,2,1,No,East
3,4,94700,1980,3,2,3,No,East
4,5,119800,2130,3,3,3,No,East


In [6]:
# se colocan nombres de columnas en minuscula
df.columns = df.columns.str.lower().str.strip()
df.columns

Index(['home', 'price', 'sqft', 'bedrooms', 'bathrooms', 'offers', 'brick',
       'neighborhood'],
      dtype='object')

###  Se deduce de la nformación, que no hay faltantes

In [7]:
# Revizamos datos outliers
df.describe().round(2)

,home,price,sqft,bedrooms,bathrooms,offers
count,128.00,128.00,128.00,128.00,128.00,128.00
mean,64.50,130427.34,2000.94,3.02,2.45,2.58
std,37.09,26868.77,211.57,0.73,0.51,1.07
min,1.00,69100.00,1450.00,2.00,2.00,1.00
25%,32.75,111325.00,1880.00,3.00,2.00,2.00
50%,64.50,125950.00,2000.00,3.00,2.00,3.00
75%,96.25,148250.00,2140.00,3.00,3.00,3.00
max,128.00,211200.00,2590.00,5.00,4.00,6.00


### No se encuentras datos que se disparen de la mayoria

In [8]:
# revizamos nulos
df.isna().sum()

,0
home,0
price,0
sqft,0
bedrooms,0
bathrooms,0
offers,0
brick,0
neighborhood,0


####No hay valores nulos

In [9]:
df["brick"].unique()

array(['No', 'Yes'], dtype=object)

In [10]:
df["neighborhood"].unique()

array(['East', 'North', 'West'], dtype=object)

In [11]:
df["price"].value_counts()

,count
price,
103200,2
105600,2
117800,2
129800,2
125700,2
...,...
119700,1
147900,1
113500,1


In [12]:
df["sqft"].value_counts()

,count
sqft,
2000,5
1920,5
1930,5
1990,4
2150,4
...,...
2530,1
1890,1
1710,1


In [13]:
df["bedrooms"].value_counts()

,count
bedrooms,
3,67
2,30
4,29
5,2


In [14]:
df["bathrooms"].value_counts()

,count
bathrooms,
2,72
3,55
4,1


#### Preparación de datos

In [15]:
df.columns

Index(['home', 'price', 'sqft', 'bedrooms', 'bathrooms', 'offers', 'brick',
       'neighborhood'],
      dtype='object')

In [16]:
# Se define el features y target.
X = df.drop(columns='price')
y = df["price"]


In [17]:
# Se dividen los datos  en train y test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [18]:
# Se selecciona columnas
num_cols = ["home", "sqft", "bedrooms", "bathrooms", "offers"]
nom_cols = ["brick", "neighborhood"]


In [19]:
# Se define transformadores.
num_transformer = Pipeline(steps=[
         ('scaler', StandardScaler())
        ])
cat_transformer = Pipeline(steps=[
         ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ])

In [20]:
# Se combina transformadores
preprocessor = ColumnTransformer(transformers=[
        ('num', num_transformer, num_cols),
        ('cat', cat_transformer, nom_cols)
        ])

In [21]:
# Se integra un pipeline
pipeline = Pipeline(steps=[
       ('preprocessor', preprocessor),
       ('model', LinearRegression())
])

In [22]:
# Se entrena el modelo
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['home', 'sqft', 'bedrooms',
                                                   'bathrooms', 'offers']),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['brick', 'neighborhood'])])),
                ('model', LinearRegression())])

In [23]:
# Se realiza predicciones
y_pred = pipeline.predict(X_test)

In [24]:
# Evaluación del modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Error Cuadrático Medio: {mse}')
print(f'R-cuadrado: {r2}')


Error Cuadrático Medio: 114251135.71732491
R-cuadrado: 0.8061565137761039


#### El modelo es relativente eficiente, puesto que maneja un 80% de predicción

### Predicción del precio de una casa :

In [25]:
df[(df["sqft"] == 2000) & (df["bedrooms"] == 3) & (df["bathrooms"] == 2)]

,home,price,sqft,bedrooms,bathrooms,offers,brick,neighborhood
21,22,113800,2000,3,2,4,No,North
113,114,115700,2000,3,2,3,Yes,North


####  Se encontro dos registros con igual cantidad de pies2 y numero de dormitorios y baños. Se saca los valores faltantes para completar un DataFrame con igual cantidad de pies2, numeros de dormitorios y baños.

#### 1.- Se completa Dataframe con valores del primer registro

In [26]:
# Se crea un DataFrame con las estructura del DataFrame del modelo con datos registro 1
casa_nueva  = pd.DataFrame({
                           'home':[22],
                           'sqft':[2000],
                       'bedrooms':[3],
                      'bathrooms':[2],
                         'offers':[4],
                          'brick':['No'],
                   'neighborhood':['North']
                 })


In [27]:
# Se realiza prediccion
precio_pred = pipeline.predict(casa_nueva)
print(f"Precio estimado casa de 2000 pies2: $ {precio_pred[0]:.2f}")

Precio estimado casa de 2000 pies2: $ 104369.85


#### 2.- Se completa Dataframe con valores del segundo registro

In [28]:
# Se crea un DataFrame con las estructura del DataFrame del modelo con datos registro 2
casa_nueva  = pd.DataFrame({
                           'home':[114],
                           'sqft':[2000],
                       'bedrooms':[3],
                      'bathrooms':[2],
                         'offers':[3],
                          'brick':['Yes'],
                   'neighborhood':['North']
                 })


In [29]:
# Se realiza predicción
precio_pred = pipeline.predict(casa_nueva)
print(f"Precio estimado casa de 2000 pies2: $ {precio_pred[0]:.2f}")

Precio estimado casa de 2000 pies2: $ 128587.30


#### 3.- Se completa dataframe con la moda de cada columna de los datos faltantes.

In [30]:
mode_home = df["home"].mode()[0]
mode_offers = df["offers"].mode()[0]
mode_brick = df["brick"].mode()[0]
mode_neighborhood = df["neighborhood"].mode()[0]
print(f" Moda de home   : {mode_home}")
print(f" Moda de offers : {mode_offers}")
print(f" Moda de brick  : {mode_brick}")
print(f" Moda de neighborhood : {mode_neighborhood}")

 Moda de home   : 1
 Moda de offers : 3
 Moda de brick  : No
 Moda de neighborhood : East


#### Se rellena el Dataset con la moda de cada columna restante

In [31]:
# Se crea DataFrame con moda de columnas
casa_nueva  = pd.DataFrame({
                           'home':[mode_home],
                           'sqft':[2000],
                       'bedrooms':[3],
                      'bathrooms':[2],
                         'offers':[mode_offers],
                          'brick':[mode_brick],
                   'neighborhood':[mode_neighborhood]
                 })


In [32]:
# Se realiza prediccion
precio_pred = pipeline.predict(casa_nueva)
print(f"Precio estimado para la casa : $ {precio_pred[0]:.2f}")

Precio estimado para la casa : $ 111746.93


#### Se concluye que usando la moda de los datos que faltan, se obtuvo un   precio de 111.746,93, más cercano a los precios reales de la muestra, para una casa de 2000 pie2 con 3 dormitorios y 2 baños.

#### Tambien, al cambiar el tipo de construcción, la diferencia de precios es notoria.
#### Precio de casa de ladrillo: 104.369,85 y casa sin ladrillo: 128.587,30.